In [1]:
import pandas as pd

In [2]:
properties_data = pd.read_csv('Sample Data/Properties - Properties.csv')
city_zip_data = pd.read_csv('Sample Data/Properties - City, Zip Codes Only - Properties.csv')

1. American Community Survey (ACS)
Why: Provides key economic and demographic information about potential customers, such as median household income, home ownership, and property values.
How: This data can be used to assess the economic capability of neighborhoods and help adjust pricing strategies accordingly.

2. Decennial Census
Why: Provides detailed data on population density and housing characteristics.
How: This data helps segment customers based on the size and type of property they own, which influences the kind of landscaping services they need and the pricing for those services.

3. Weather Data (External)
Why: Landscaping is highly dependent on climate, and different weather patterns will influence the frequency and intensity of services required.
How: Using external weather data will help forecast seasonal demand and optimize pricing based on service needs throughout the year.


In [3]:
df = properties_data.copy()
df

,Property Name,Branch Name,Industry Name,Irrigation Controller Count,Irrigation Controller Type,Irrigation Flow Sensor,Irrigation Master Valve,Irrigation Meter Count,Lead Source,Property Address Line 1,Property City,Property Group Name,Property State/Province Code,Property Type,Property Zip Code,Turf SF,Planter SF,Slope SF
0,10917 Cherry Ave bt41025,Bemus Riverside,Commercial,Unknown,Unknown,Unknown,Unknown,Unknown,Business Development,10917 Cherry Ave,Fontana,Bemus- Maintenance/Tree Care,CA,Maintenance and Tree Care,92337,0.0,26233.0,0.0
1,11553 Foothill Blvd. RC,Bemus Riverside,Commercial,1,Smart,No,No,Unknown,Customer Referral,11553 Foothill Blvd,Rancho Cucamonga,Bemus- Maintenance/Tree Care,CA,Maintenance and Tree Care,91730,0.0,6587.0,0.0
2,14144 Santa Ana Ave bt41119,Bemus Riverside,Commercial,Unknown,Unknown,Unknown,Unknown,Unknown,Business Development,14144 Santa Ana Ave,Fontana,Bemus- Maintenance/Tree Care,CA,Maintenance and Tree Care,92337,0.0,36905.0,0.0
3,1660 N Linden - BR78102,Bemus Riverside,Commercial,2,Smart,Unknown,Unknown,Unknown,Business Development,1660 N Linden Ave,Rialto,Bemus- Maintenance/Tree Care,CA,Maintenance and Tree Care,92376,0.0,162166.0,133260.0
4,1675 W Park Ave bt41053,Bemus Riverside,Commercial,Unknown,Unknown,Unknown,Unknown,Unknown,Business Development,1675 W Park Ave,Redlands,Bemus- Maintenance/Tree Care,CA,Maintenance and Tree Care,92373,0.0,44382.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5527,Zante Village HOA,Bemus San Marcos,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Pindar Way,Oceanside,Bemus- Tree Care,CA,Tree Care,92056,NaN,NaN,NaN
5528,Ziani,Bemus San Marcos,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,2400 Torrey Pines Road,La Jolla,Unknown,CA,Unknown,92037,NaN,NaN,NaN
5529,Ziani Homeowner Association,Bemus Santa Ana - Anne,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,1 Saraceno,Newport Beach,Unknown,CA,Unknown,92657,NaN,NaN,NaN
5530,Zinnia Neighborhood,Bemus Santa Ana - Hathaway,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Customer Referral,95 Tallowood,Irvine,Bemus- Maintenance/Tree Care,CA,Maintenance and Tree Care,92620,NaN,NaN,NaN


In [10]:
import requests
import time

# Function to get ACS data for a given zip code
def get_acs_data(zip_code, api_key):
    url = f"https://api.census.gov/data/2020/acs/acs5?get=NAME,B19013_001E,B25003_002E,B25077_001E&for=zip%20code%20tabulation%20area:{zip_code}&key={api_key}"
    
    # Print the URL and ZIP code to debug
    print(f"Requesting data for ZIP code: {zip_code}")
    
    response = requests.get(url)
    
    if response.status_code == 200:
        try:
            data = response.json()
            # Extract relevant data: Median Income, Home Ownership, Median Home Value
            median_income = data[1][1]
            home_ownership = data[1][2]
            median_home_value = data[1][3]
            return median_income, home_ownership, median_home_value
        except (IndexError, ValueError, KeyError) as e:
            print(f"Error fetching data for ZIP code {zip_code}: {e}")
            return None, None, None
    else:
        print(f"Failed to fetch data for ZIP code {zip_code}: {response.status_code}")
        return None, None, None

# Replace with your actual Census API key
census_api_key = '5ce2fbb1b20a6e18d8a0601513eac2032a237512'

# Add new columns to the dataframe for ACS data
properties_data['Median Income'] = None
properties_data['Home Ownership'] = None
properties_data['Median Home Value'] = None

# Test on a subset of rows first (e.g., first 10 rows)
subset = properties_data.head(10)

# Loop through each ZIP code in the subset and fetch ACS data
for idx, row in subset.iterrows():
    zip_code = str(row['Property Zip Code']).zfill(5)  # Ensure the ZIP code is in 5-digit format
    median_income, home_ownership, median_home_value = get_acs_data(zip_code, census_api_key)
    
    # Store the results in the dataframe
    properties_data.at[idx, 'Median Income'] = median_income
    properties_data.at[idx, 'Home Ownership'] = home_ownership
    properties_data.at[idx, 'Median Home Value'] = median_home_value
    
    # Delay the requests to avoid rate limits (adjust this as necessary)
    time.sleep(1)

# Display the updated dataframe
print(properties_data.head())


Requesting data for ZIP code: 92337
Requesting data for ZIP code: 91730
Requesting data for ZIP code: 92337
Requesting data for ZIP code: 92376
Requesting data for ZIP code: 92373
Requesting data for ZIP code: 92570
Requesting data for ZIP code: 91761
Requesting data for ZIP code: 92880
Requesting data for ZIP code: 92376
Requesting data for ZIP code: 92408
                 Property Name      Branch Name Industry Name  \
0     10917 Cherry Ave bt41025  Bemus Riverside    Commercial   
1      11553 Foothill Blvd. RC  Bemus Riverside    Commercial   
2  14144 Santa Ana Ave bt41119  Bemus Riverside    Commercial   
3      1660 N Linden - BR78102  Bemus Riverside    Commercial   
4      1675 W Park Ave bt41053  Bemus Riverside    Commercial   

  Irrigation Controller Count Irrigation Controller Type  \
0                     Unknown                    Unknown   
1                           1                      Smart   
2                     Unknown                    Unknown   
3        

In [11]:
properties_data

,Property Name,Branch Name,Industry Name,Irrigation Controller Count,Irrigation Controller Type,Irrigation Flow Sensor,Irrigation Master Valve,Irrigation Meter Count,Lead Source,Property Address Line 1,...,Property Group Name,Property State/Province Code,Property Type,Property Zip Code,Turf SF,Planter SF,Slope SF,Median Income,Home Ownership,Median Home Value
0,10917 Cherry Ave bt41025,Bemus Riverside,Commercial,Unknown,Unknown,Unknown,Unknown,Unknown,Business Development,10917 Cherry Ave,...,Bemus- Maintenance/Tree Care,CA,Maintenance and Tree Care,92337,0.0,26233.0,0.0,84649,7417,373600
1,11553 Foothill Blvd. RC,Bemus Riverside,Commercial,1,Smart,No,No,Unknown,Customer Referral,11553 Foothill Blvd,...,Bemus- Maintenance/Tree Care,CA,Maintenance and Tree Care,91730,0.0,6587.0,0.0,74943,11771,427400
2,14144 Santa Ana Ave bt41119,Bemus Riverside,Commercial,Unknown,Unknown,Unknown,Unknown,Unknown,Business Development,14144 Santa Ana Ave,...,Bemus- Maintenance/Tree Care,CA,Maintenance and Tree Care,92337,0.0,36905.0,0.0,84649,7417,373600
3,1660 N Linden - BR78102,Bemus Riverside,Commercial,2,Smart,Unknown,Unknown,Unknown,Business Development,1660 N Linden Ave,...,Bemus- Maintenance/Tree Care,CA,Maintenance and Tree Care,92376,0.0,162166.0,133260.0,59416,12933,309700
4,1675 W Park Ave bt41053,Bemus Riverside,Commercial,Unknown,Unknown,Unknown,Unknown,Unknown,Business Development,1675 W Park Ave,...,Bemus- Maintenance/Tree Care,CA,Maintenance and Tree Care,92373,0.0,44382.0,0.0,81557,6730,485300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5527,Zante Village HOA,Bemus San Marcos,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Pindar Way,...,Bemus- Tree Care,CA,Tree Care,92056,NaN,NaN,NaN,None,None,None
5528,Ziani,Bemus San Marcos,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,2400 Torrey Pines Road,...,Unknown,CA,Unknown,92037,NaN,NaN,NaN,None,None,None
5529,Ziani Homeowner Association,Bemus Santa Ana - Anne,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,1 Saraceno,...,Unknown,CA,Unknown,92657,NaN,NaN,NaN,None,None,None
5530,Zinnia Neighborhood,Bemus Santa Ana - Hathaway,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Customer Referral,95 Tallowood,...,Bemus- Maintenance/Tree Care,CA,Maintenance and Tree Care,92620,NaN,NaN,NaN,None,None,None


In [12]:
top_10_properties = properties_data.head(10)
top_10_properties.to_csv('top_10_properties.csv', index=False)